In [5]:
from ga4gh.vrs import models, vrs_deref, vrs_enref
from ga4gh.core import ga4gh_identify, ga4gh_serialize, ga4gh_digest, ga4gh_deref

import json
def ppo(o, indent=2):
    """pretty print object as json"""
    print(json.dumps(o.as_dict(), sort_keys=True, indent=indent))
    
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy
from biocommons.seqrepo import SeqRepo
from ga4gh.vrs.extras.translator import Translator

In [8]:
seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"
dp = SeqRepoRESTDataProxy(base_url=seqrepo_rest_service_url)

tlr = Translator(data_proxy=dp)

## Representing ClinVar PGx Allele 634420
https://www.ncbi.nlm.nih.gov/clinvar/variation/634420

In [4]:
# "Haplotype" 1: https://www.ncbi.nlm.nih.gov/clinvar/variation/633851

In [6]:
hgvs_expression = "NC_000010.11:g.94761665G>A"

In [12]:
h1_allele1 = tlr.translate_from(hgvs_expression,'hgvs')

In [13]:
# Haplotype 2: https://www.ncbi.nlm.nih.gov/clinvar/variation/633860/
# H2 Allele 1: https://www.ncbi.nlm.nih.gov/clinvar/variation/16900
# H2 Allele 2: https://www.ncbi.nlm.nih.gov/clinvar/variation/39357

hgvs_expression = "NC_000010.11:g.94762706A>G"
h2_allele1 = tlr.translate_from(hgvs_expression,'hgvs')

hgvs_expression = "NC_000010.11:g.94761900C>T"
h2_allele2 = tlr.translate_from(hgvs_expression,'hgvs')

In [15]:
h2 = models.Haplotype(members=[h2_allele1, h2_allele2])

In [ ]:
models.Number(value=1)

<Number type=None value=<Literal<int> 1>>

In [21]:
gt_mem1 = models.GenotypeMember(variation=h1_allele1, copies=models.Number(value=1))

ValidationError: <class 'abc.Allele'>: __new__() takes 1 positional argument but 2 were given
Unable to instantiate any valid types: 
<class 'abc.Haplotype'>: __new__() takes 1 positional argument but 2 were given
 
while setting 'variation' in GenotypeMember

In [ ]:
gt_mem2 = models.GenotypeMember(variation=h2, copies=models.Number(value=1))
gt = models.Genotype(members=[gt_mem1, gt_mem2], copies=models.Number(value=2))

In [17]:
h1_allele1

<Allele _id=<Literal<str> ga4gh:VA.3AIu_q8I6dxuX76aZIq43GvogEsLuvAN> location=<SequenceLocation _id=None interval=<SequenceInterval end=<Number type=<Literal<str> Number> value=<Literal<int> 94761665>> start=<Number type=<Literal<str> Number> value=<Literal<int> 94761664>> type=<Literal<str> SequenceInterval>> sequence_id=<Literal<str> ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB> type=<Literal<str> SequenceLocation>> state=<SequenceState sequence=<Literal<str> A> type=<Literal<str> LiteralSequenceExpression>> type=<Literal<str> Allele>>

In [26]:
sl1 = h1_allele1.location
ppo(models.DerivedSequenceExpression(location=sl1, reverse_complement=False))

{
  "location": {
    "interval": {
      "end": {
        "type": "Number",
        "value": 94761665
      },
      "start": {
        "type": "Number",
        "value": 94761664
      },
      "type": "SequenceInterval"
    },
    "sequence_id": "ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB",
    "type": "SequenceLocation"
  },
  "reverse_complement": false
}
